# Zagadki

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to zadanie możesz zdobyć pomiędzy 0 i 1.5 punktu. Zdobędziesz 0 punktów jeśli wartość kryterium `mean reciprocal rank` na zbiorze testowym wyniesie poniżej 0.02, a 1.5 punktu jeśli wyniesie powyżej 0.3. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością kryterium.

# Kod startowy

In [1]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [2]:

!pip install --upgrade numpy gensim

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)


In [3]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm

## Ładowanie danych

In [4]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
path_to_data = 'data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

##Ta funkcja służy do sprowadzenia słowa do jego podstawowej (kanonicznej) formy, czyli tzw. lematyzacji.

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)
    if ret:
        return ret
    return word

## Pobranie danych

In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

Retrieving folder contents


Processing file 1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI plwiktionary_definitions_clean.txt
Processing file 1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk superbazy_clean.txt
Processing file 1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh w2v_polish_lemmas.model
Processing file 1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC w2v_polish_lemmas.model.syn1neg.npy
Processing file 1RY0Ftfx_-nPUbddYCvHq9p8yCcXbUrYS w2v_polish_lemmas.model.wv.vectors.npy
Processing file 18wrF9VyESTvIT9Z_TqUd_2wV4lP2As3t zagadki_do_testow_clean.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1dCYKzdg6TihyckNM9Zxr1g9lWT1BzDXI
To: /content/data/zagadki/plwiktionary_definitions_clean.txt
100%|██████████| 1.59M/1.59M [00:00<00:00, 59.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1q6Ki5Y66gugM30oFcCtJj7ocMJymskSk
To: /content/data/zagadki/superbazy_clean.txt
100%|██████████| 43.7M/43.7M [00:00<00:00, 84.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GZPNIR16bxFnzvbIYDLacn8TcKwiGnIh
To: /content/data/zagadki/w2v_polish_lemmas.model
100%|██████████| 40.8M/40.8M [00:00<00:00, 58.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC
From (redirected): https://drive.google.com/uc?id=1C-V5TgIAHUJp_FLD-bvks6LmMkrfQpzC&confirm=t&uuid=24b28e28-d660-4812-b936-c0a7d65f411d
To: /content/data/zagadki/w2v_polish_lemmas.model.syn1neg.npy
100%|██████████| 958M/958M

## Wczytywanie lematów do słownika bases

In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

## Ładowanie modelu Word2Vec

In [8]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
model = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

## Wczytywanie definicji słów + wstępne IDF

In [9]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

nltk.download('punkt_tab')
for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    L = word.split()
    if len(L) == 1:
        word = L[0]

        definition = set(tokenize(definition.lower()))
        all_word_definitions[word].append(definition)
        for word in set(definition):
            base_idf[get_word_base(word)] += 1


for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Wczytywanie zagadek i odpowiedzi

In [10]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()
        answers.append(line[0])
        queries.append(tokenize(' '.join(line[1:])))

## Kod z kryteriami oceniającymi

In [11]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)

    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [19]:
import time
import math
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

# ------------------------
# Stałe i słowa stopu
# ------------------------

STOP_WORDS = [ "a", "aby", "ach", "acz", "aczkolwiek", "aj", "albo", "ale", "ależ", "ani", "aż", "bardziej", "bardzo",
               "bez", "bo", "bowiem", "by", "byli", "bym", "bynajmniej", "być", "był", "była", "było", "były", "będzie",
               "będą", "cali", "cała", "cały", "chce", "choć", "ci", "ciebie", "cię", "co", "cokolwiek", "coraz", "coś",
               "czasami", "czasem", "czemu", "czy", "czyli", "często", "daleko", "dla", "dlaczego", "dlatego", "do",
               "dobrze", "dokąd", "dość", "dr", "dużo", "dwa", "dwaj", "dwie", "dwoje", "dzisiaj", "dziś", "gdy",
               "gdyby", "gdyż", "gdzie", "gdziekolwiek", "gdzieś", "go", "godz", "hab", "i", "ich", "ii", "iii", "ile",
               "im", "inna", "inne", "inny", "innych", "inż", "iv", "ix", "iż", "ja", "jak", "jakaś", "jakby", "jaki",
               "jakichś", "jakie", "jakiś", "jakiż", "jakkolwiek", "jako", "jakoś", "je", "jeden", "jedna", "jednak",
               "jednakże", "jedno", "jednym", "jedynie", "jego", "jej", "jemu", "jest", "jestem", "jeszcze", "jeśli",
               "jeżeli", "już", "ją", "każdy", "kiedy", "kierunku", "kilka", "kilku", "kimś", "kto", "ktokolwiek",
               "ktoś", "która", "które", "którego", "której", "który", "których", "którym", "którzy", "ku", "lat", "lecz",
               "lub", "ma", "mają", "mam", "mamy", "mało", "mgr", "mi", "miał", "mimo", "między", "mnie", "mną", "mogą",
               "moi", "moim", "moja", "moje", "może", "możliwe", "można", "mu", "musi", "my", "mój", "na", "nad", "nam",
               "nami", "nas", "nasi", "nasz", "nasza", "nasze", "naszego", "naszych", "natomiast", "natychmiast",
               "nawet", "nic", "nich", "nie", "niech", "niego", "niej", "niemu", "nigdy", "nim", "nimi", "nią", "niż",
               "no", "nowe", "np", "nr", "o", "o.o.", "obok", "od", "ok", "około", "on", "ona", "one", "oni", "ono",
               "oraz", "oto", "owszem", "pan", "pana", "pani", "pl", "po", "pod", "podczas", "pomimo", "ponad",
               "ponieważ", "powinien", "powinna", "powinni", "powinno", "poza", "prawie", "prof", "przecież", "przed",
               "przede", "przedtem", "przez", "przy", "raz", "razie", "roku", "również", "sam", "sama", "się", "skąd",
               "sobie", "sobą", "sposób", "swoje", "są", "ta", "tak", "taka", "taki", "takich", "takie", "także", "tam",
               "te", "tego", "tej", "tel", "temu", "ten", "teraz", "też", "to", "tobie", "tobą", "toteż", "trzeba",
               "tu", "tutaj", "twoi", "twoim", "twoja", "twoje", "twym", "twój", "ty", "tych", "tylko", "tym", "tys",
               "tzw", "tę", "u", "ul", "vi", "vii", "viii", "vol", "w", "wam", "wami", "was", "wasi", "wasz", "wasza",
               "wasze", "we", "według", "wie", "wiele", "wielu", "więc", "więcej", "wszyscy", "wszystkich", "wszystkie",
               "wszystkim", "wszystko", "wtedy", "www", "wy", "właśnie", "wśród", "xi", "xii", "xiii", "xiv", "xv", "z",
               "za", "zapewne", "zawsze", "zaś", "ze", "znowu", "znów", "został", "zł", "żaden", "żadna", "żadne",
               "żadnych", "że", "żeby"]

# ------------------------
# Przygotowanie danych
# ------------------------

idf_words = {}
word_freq = {}
total_word_count = 0
all_words = set()
num_sentences = 0

for definitions in all_word_definitions:
    for sentence in all_word_definitions[definitions]:
        for word in sentence:
            base_word = get_word_base(word)
            word_freq[base_word] = word_freq.get(base_word, 0) + 1
            total_word_count += 1
            all_words.add(base_word)
        num_sentences += 1

# ------------------------
# SIF Weights
# ------------------------

a = 0.001
sif_weights = {word: a / (a + (word_freq.get(word, 0) / total_word_count + 1e-8)) for word in all_words}

# ------------------------
# Szybsze usuwanie śmieci
# ------------------------

trash_table = str.maketrans('', '', ";'{}.,\n()_")

def remove_trash(word):
    return word.translate(trash_table)

# ------------------------
# Embedding bez PCA
# ------------------------

def sentence_embedding_no_pca(sentence):
    avg_vector = np.zeros(200)
    valid_word_count = 0

    for word in sentence:
        word = remove_trash(word)
        word = get_word_base(word)
        if word not in STOP_WORDS:
            vector = model.wv[word] * sif_weights.get(word, a) if word in model.wv else np.zeros(200)
            avg_vector += vector
            valid_word_count += 1

    return avg_vector / valid_word_count if valid_word_count > 0 else avg_vector

# ------------------------
# Wyliczanie sentence_vectors
# ------------------------

sentence_vectors = []

for definitions in all_word_definitions:
    for sentence in all_word_definitions[definitions]:
        embedding = sentence_embedding_no_pca(sentence)
        sentence_vectors.append(embedding)

sentence_vectors = np.array(sentence_vectors)

# ------------------------
# PCA (z wieloma komponentami)
# ------------------------

svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(sentence_vectors)
pcs = svd.components_

# ------------------------
# PCA - usuwanie komponentów
# ------------------------

def remove_pc(vector, pcs, beta=0.7):
    v = vector.copy()
    for pc in pcs:
        v -= beta * np.dot(pc, v) * pc
    return v

# ------------------------
# Finalne embeddingi
# ------------------------

def sentence_embedding(sentence):
    avg_vector = sentence_embedding_no_pca(sentence)
    vector = remove_pc(avg_vector, pcs, beta=0.7)
    return vector / np.linalg.norm(vector) if np.linalg.norm(vector) > 0 else vector

# ------------------------
# Embedding definicji
# ------------------------

all_def_embeddings = [
    (definition, sentence_embedding(sentence))
    for definition in all_word_definitions
    for sentence in all_word_definitions[definition]
]

# ------------------------
# Metryka cosine similarity
# ------------------------

def cosine_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-8)

# ------------------------
# Odpowiadanie na zagadkę
# ------------------------

def answer_riddle(riddle, K):
    riddle_vector = sentence_embedding(riddle)
    distances = []

    for definition, definition_vector in all_def_embeddings:
        sim = cosine_sim(riddle_vector, definition_vector)
        distances.append((sim, definition))

    distances.sort(reverse=True)  # większe = lepsze
    return [definition for _, definition in distances[:K]]

In [13]:
import time
import math
import numpy as np

# ------------------------
# Stałe i słowa stopu
# ------------------------

STOP_WORDS = [ "a","aby","ach","acz","aczkolwiek","aj","albo","ale","ależ","ani","aż","bardziej","bardzo",
               "bez","bo","bowiem","by","byli","bym","bynajmniej","być","był","była","było","były","będzie",
               "będą","cali","cała","cały","chce","choć","ci","ciebie","cię","co","cokolwiek","coraz","coś",
               "czasami","czasem","czemu","czy","czyli","często","daleko","dla","dlaczego","dlatego","do",
               "dobrze","dokąd","dość","dr","dużo","dwa","dwaj","dwie","dwoje","dzisiaj","dziś","gdy",
               "gdyby","gdyż","gdzie","gdziekolwiek","gdzieś","go","godz","hab","i","ich","ii","iii","ile",
               "im","inna","inne","inny","innych","inż","iv","ix","iż","ja","jak","jakaś","jakby","jaki",
               "jakichś","jakie","jakiś","jakiż","jakkolwiek","jako","jakoś","je","jeden","jedna","jednak",
               "jednakże","jedno","jednym","jedynie","jego","jej","jemu","jest","jestem","jeszcze","jeśli",
               "jeżeli","już","ją","każdy","kiedy","kierunku","kilka","kilku","kimś","kto","ktokolwiek",
               "ktoś","która","które","którego","której","który","których","którym","którzy","ku","lat","lecz",
               "lub","ma","mają","mam","mamy","mało","mgr","mi","miał","mimo","między","mnie","mną","mogą",
               "moi","moim","moja","moje","może","możliwe","można","mu","musi","my","mój","na","nad","nam",
               "nami","nas","nasi","nasz","nasza","nasze","naszego","naszych","natomiast","natychmiast",
               "nawet","nic","nich","nie","niech","niego","niej","niemu","nigdy","nim","nimi","nią","niż",
               "no","nowe","np","nr","o","o.o.","obok","od","ok","około","on","ona","one","oni","ono",
               "oraz","oto","owszem","pan","pana","pani","pl","po","pod","podczas","pomimo","ponad",
               "ponieważ","powinien","powinna","powinni","powinno","poza","prawie","prof","przecież","przed",
               "przede","przedtem","przez","przy","raz","razie","roku","również","sam","sama","się","skąd",
               "sobie","sobą","sposób","swoje","są","ta","tak","taka","taki","takich","takie","także","tam",
               "te","tego","tej","tel","temu","ten","teraz","też","to","tobie","tobą","toteż","trzeba",
               "tu","tutaj","twoi","twoim","twoja","twoje","twym","twój","ty","tych","tylko","tym","tys",
               "tzw","tę","u","ul","vi","vii","viii","vol","w","wam","wami","was","wasi","wasz","wasza",
               "wasze","we","według","wie","wiele","wielu","więc","więcej","wszyscy","wszystkich","wszystkie",
               "wszystkim","wszystko","wtedy","www","wy","właśnie","wśród","xi","xii","xiii","xiv","xv","z",
               "za","zapewne","zawsze","zaś","ze","znowu","znów","został","zł","żaden","żadna","żadne",
               "żadnych","że","żeby"]

# ------------------------
# Przygotowanie danych
# ------------------------

idf_words = {}
word_freq = {}
total_word_count = 0
all_words = []
num_sentences = 0

for definitions in all_word_definitions:
    for sentence in all_word_definitions[definitions]:
        for word in sentence:
            base_word = get_word_base(word)
            word_freq[base_word] = word_freq.get(base_word, 0) + 1
            total_word_count += 1
            if base_word not in all_words:
                all_words.append(base_word)
        num_sentences += 1

a = 0.001
sif_weights = {word: a / (a + (word_freq.get(word, 0) / total_word_count + 1e-8)) for word in all_words}

for word in all_words:
    idf_words[word] = 0

for definitions in all_word_definitions:
    for sentence in all_word_definitions[definitions]:
        added = []
        for word in sentence:
            base_word = get_word_base(word)
            if base_word not in added:
                idf_words[base_word] += 1
                added.append(base_word)

for word in all_words:
    idf_words[word] = math.log(num_sentences / idf_words[word], 2)

idf_average = sum(idf_words.values()) / len(idf_words)

# ------------------------
# Funkcje pomocnicze
# ------------------------

def remove_trash(word):
    to_remove = [';', "'", '{', '}', '.', ',', '\n', '(', ')', '_']
    for char in to_remove:
        word = word.replace(char, '')
    return word

# ------------------------
# Funkcja Sentence_Embeding (bez PCA i z normalizacją)
# ------------------------

def Sentence_Embeding(sentence):
    avg_vector = np.zeros(200)
    valid_word_count = 0
    for word in sentence:
        word = remove_trash(word)
        word = get_word_base(word)
        if word not in STOP_WORDS:
            try:
                vector = model.wv[word] * sif_weights.get(word, a)
            except Exception as e:
                try:
                    vector = model.wv[word] * idf_average
                except Exception as a:
                    vector = np.zeros(200)
            avg_vector += vector
            valid_word_count += 1
    if valid_word_count > 0:
        avg_vector /= valid_word_count
    # Normalizacja wektora
    norm = np.linalg.norm(avg_vector)
    if norm > 0:
        avg_vector = avg_vector / norm
    return avg_vector

# ------------------------
# Generowanie embeddingów definicji (bez PCA)
# ------------------------

all_def_embedings = []
for x in all_word_definitions:
    for y in all_word_definitions[x]:
        all_def_embedings.append((x, Sentence_Embeding(y)))

# ------------------------
# Metryka cosine similarity
# ------------------------

def cosine_sim(v1, v2):
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)
    if norm1 == 0 or norm2 == 0:
        return 0
    return np.dot(v1, v2) / (norm1 * norm2)

# ------------------------
# Odpowiadanie na zagadkę z metryką cosine similarity
# ------------------------

def answer_riddle(riddle, K):
    riddle_vector = Sentence_Embeding(riddle)
    distances = []
    for definition, definition_vector in all_def_embedings:
        sim = cosine_sim(riddle_vector, definition_vector)
        distances.append((sim, definition))
    distances.sort(key=lambda x: x[0], reverse=True)  # Wyższa podobieństwo = lepsze
    ans = [definition for _, definition in distances[:K]]
    return ans


# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [14]:
import pandas as pd

# Zmodyfikowana funkcja evaluate_algorithm
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    data_for_csv = []

    for query, real_answer in zip(queries, answers):
        # Generujemy odpowiedzi za pomocą naszego algorytmu
        predicted_answers = score_function(set(query), K=K)

        # Obliczamy punkty za to pytanie
        if real_answer in predicted_answers[:K]:
            pos = predicted_answers.index(real_answer) + 1
            score = 1 / pos
        else:
            score = 0

        # Zbieramy dane do zapisania: pytanie, prawidłowa odpowiedź, 20 najprawdopodobniejszych odpowiedzi, punkty
        data_for_csv.append({
            'Pytanie': ' '.join(query),  # Zagadka jako tekst
            'Prawidłowa odpowiedź': real_answer,
            '20 Najbardziej prawdopodobnych odpowiedzi': ', '.join(predicted_answers[:K]),
            'Punkty': score
        })

        computed_answers.append(predicted_answers)

    # Zapisywanie wyników do CSV
    df = pd.DataFrame(data_for_csv)
    df.to_excel('wyniki.xlsx', index=False)


    # Obliczamy Mean Reciprocal Rank
    mrr = mean_reciprocal_rank(answers, computed_answers, K=K)
    print ('Mean Reciprocal Rank =', mrr)

    return mrr



In [20]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    PART_OF_DATA = 100
    K = 20
    valid_queries = queries[:PART_OF_DATA]
    valid_answers = answers[:PART_OF_DATA]
    score = evaluate_algorithm(answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

Mean Reciprocal Rank = 0.24584074155978178
Mean Reciprocal Rank = 0.24584074155978178
Score: 0.24584074155978178


In [21]:
# Wyświetlenie przykładowych danych z plików

import random

# Wyświetlenie 5 przykładowych zagadek
print("===== Przykładowe zagadki (zagadki_do_testow_clean.txt) =====\n")
with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt', encoding='utf-8') as f:
    lines = f.readlines()
    for line in random.sample(lines, 5):
        answer, *riddle = line.strip().replace(';;','').split()
        print(f"Zagadka: {' '.join(riddle)}\nOdpowiedź: {answer}\n")

# Wyświetlenie 5 przykładowych definicji
print("\n===== Przykładowe definicje słów (plwiktionary_definitions_clean.txt) =====\n")
with open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt', encoding='utf-8') as f:
    lines = f.readlines()
    for line in random.sample(lines, 5):
        word, definition = line.strip().split("###")
        print(f"Słowo: {word}\nDefinicja: {definition}\n")

# Wyświetlenie 5 przykładowych bazowych form słów
print("\n===== Przykładowe formy bazowe (superbazy_clean.txt) =====\n")
with open(f'{path_to_data}/zagadki/superbazy_clean.txt', encoding='utf-8') as f:
    lines = f.readlines()
    for line in random.sample(lines, 5):
        word, base = line.strip().split()
        print(f"Słowo odmienione: {word}\nForma bazowa: {base}\n")


===== Przykładowe zagadki (zagadki_do_testow_clean.txt) =====

Zagadka: tytuł noszony przez zwierzchnika zakonu kawalerów mieczowych lub zakonu krzyżackiego.
Odpowiedź: komtur

Zagadka: proces lub efekt oddzielania lub rozdzielania czegoś na części lub elementy.
Odpowiedź: separacja

Zagadka: naturalne pragnienie jedzenia, stanowi regulację podaż i spożycie pokarmów potrzebnych do utrzymania zdrowia i życia.
Odpowiedź: apetyt

Zagadka: punkt na powierzchni ziemi bezpośrednio nad miejscem, gdzie doszło do trzęsienia ziemi.
Odpowiedź: epicentrum

Zagadka: organizm żywy, który poluje na inne zwierzęta w celu zdobycia pożywienia.
Odpowiedź: drapieżnik


===== Przykładowe definicje słów (plwiktionary_definitions_clean.txt) =====

Słowo: oprawa 
Definicja:  okładka książki, zeszytu itp.

Słowo: marka 
Definicja:  znak

Słowo: opieka 
Definicja:  dbanie o kogoś lub coś; {{wikipedia}}

Słowo: zona 
Definicja:  ''3. {{os}} {{lp}} czasu teraźniejszego trybu oznajmującego (presente do indicativo)

In [22]:
# Wczytanie pliku
df = pd.read_excel('wyniki.xlsx')

# Podstawowe liczenie
liczba_z_punktami = (df['Punkty'] > 0).sum()
liczba_z_jednym_punktem = (df['Punkty'] == 1).sum()
liczba_bez_punktow = (df['Punkty'] == 0).sum()

print(f'Liczba wierszy bez punktów: {liczba_bez_punktow}')
print(f'Liczba wierszy z jakimikolwiek punktami (>0): {liczba_z_punktami}')
print(f'Liczba wierszy z dokładnie 1 punktem: {liczba_z_jednym_punktem}')

print(f'Łączna liczba wierszy: {len(df)}')
print(f'Odsetek idealnie trafionych (1 punkt): {liczba_z_jednym_punktem / len(df) * 100:.2f}%')
print(f'Odsetek zagadek z jakimikolwiek punktami: {liczba_z_punktami / len(df) * 100:.2f}%')


Liczba wierszy bez punktów: 46
Liczba wierszy z jakimikolwiek punktami (>0): 54
Liczba wierszy z dokładnie 1 punktem: 15
Łączna liczba wierszy: 100
Odsetek idealnie trafionych (1 punkt): 15.00%
Odsetek zagadek z jakimikolwiek punktami: 54.00%
